In [1]:
# default_exp signUp

# Sign up

> sign up

In [12]:
#export
###################### Imports ###########################
import hashlib, uuid, os, logging, sys
import ujson as json
from awsSchema.apigateway import Event,Response
from src.Thread import Thread
from beartype import beartype
from copy import deepcopy
from pynamodb.models import Model
from pynamodb.attributes import (
    UnicodeAttribute, NumberAttribute, UnicodeSetAttribute, UTCDateTimeAttribute
)

In [3]:
#export
############### Logger for debugging code ##################
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [4]:
#export
########## Error Definitions ##########
class HelperError(Exception): pass
class ParseInputError(HelperError): pass
class AddUserError(HelperError): pass
class CreateTableError(HelperError): pass
class UsernameAvailabilityError(HelperError): pass

In [6]:
#hide
os.environ['USERPASSWORDTABLE'] = 'user-password-demo-sallee-master'

In [7]:
#export
################ Setting Globals from Env Vars ################
USERPASSWORDTABLE = os.environ['USERPASSWORDTABLE']

In [9]:
#export
########## Helper class for main function ##########
EventInput = dict
class H:
    @staticmethod
    @beartype
    def sha256(password: str) -> str:
        return hashlib.sha256(password.encode()).hexdigest()
    
    @staticmethod
    @beartype
    def salt() -> str:
        return uuid.uuid4().hex

    @classmethod
    @beartype
    def salted_sha256(cls, password: str, salt: str ='') -> tuple:
        if salt == '':
            salt = cls.salt()
        return f'{cls.sha256(salt + password)}', f'{salt}'
    
    @staticmethod
    @beartype
    def add_user_to_table(username: str, hash: str, salt: str, hashAndSalt: str):
        try:
            userTable = os.environ['USERPASSWORDTABLE']
            threadItem = Thread(username=deepcopy(username), 
                                passwordHash=deepcopy(hash), 
                                salt=deepcopy(salt), 
                                hashAndSalt=deepcopy(hashAndSalt))
            threadItem.save()
        except Exception as e:
            logger.error(f'Unable to add user to the database:\n{e}')
            raise AddUserError(f'Unable to add user to the database:\n{e}')
        
    @staticmethod
    @beartype
    def parseInput(event: EventInput) -> tuple:
        '''
        returns username and password arguments from input
        '''
        body = Event.parseBody(deepcopy(event))
        try:
            username = body['username']
        except KeyError:
            logger.error('username is not in body')
            raise ParseInputError('username is not in body')

        try:
            password = body['password']
        except KeyError:
            logger.error('password is not in body')
            raise ParseInputError('password is not in body')

        return username, password
    
    @staticmethod
    def createTable():
        '''Cretaes the table if it doesn't exist'''

        try:
            if not Thread.exists():
                Thread.create_table(billing_mode='PAY_PER_REQUEST')
        except Exception as e:
            logger.error(f'Unable to create database:\n{e}')
            raise CreateTableError(f'Unable to create database:\n{e}')
    
    @staticmethod
    @beartype
    def usernameAvailable(username: str) -> bool:
        try:
            queryResult = Thread.query(username)
            listResult = [row for row in queryResult]
            if len(listResult) > 0:
                return False
            return True
        except Exception as e:
            logger.error(f'Unable to check whether or not the username is available:\n{e}')
            raise UsernameAvailabilityError(f'Unable to check whether or not the username is available:\n{e}')
        

In [10]:
#export
############################## Main Function ###############################
def signUp(event, *args):

  logger.info(f"Password table name :: {USERPASSWORDTABLE}")
  
  evtCpy = deepcopy(event)
  logger.info(f'Event :: {evtCpy}')

  H.createTable()

  username, password = H.parseInput(evtCpy)
  # Take this away before using, it isn't a good idea to save the username and pw in logs
  # logger.info(f"Username :: {username}\npassword :: {password}")

  hashedPw, salt = H.salted_sha256(password)
  hashAndSalt = hashedPw + ':' + salt
  logger.info(f'Hashed Pass :: {hashedPw}')
  logger.info(f'Salt :: {salt}')

  if H.usernameAvailable(username):
    H.add_user_to_table(username, hashedPw, salt, hashAndSalt)
  
  return Response.returnSuccess("Success")

In [16]:
#hide
sampleInput = {'username' : 'Samuel', 'password' : '1234'}
sampleEvent = Event.getInput(body=sampleInput)
print(json.dumps(sampleEvent))


{"body":"{\"username\":\"Samuel\",\"password\":\"1234\"}","httpMethod":null,"multiValueHeaders":null,"multiValueQueryStringParameters":null,"path":null,"pathParameters":null,"queryStringParameters":{},"requestContext":null,"headers":{},"statusCode":200,"isBase64Encoded":false}


In [19]:
#hide
print(type(H.salt()))

<class 'str'>
